In [1]:
from libs.alibaba.alibaba import Alibaba

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq
from libs.CeleryTasks import tasks
import html
import time
import re
import traceback
import types

In [80]:
import pendulum

In [2]:
lid = 'eyelash@glittergroupcn.com'
lpwd = 'Wangzhen68970747'
alibaba_eyelash = Alibaba(lid, lpwd)
alibaba_eyelash.login()

primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Jeremy


In [16]:
url = 'https://hz-mydata.alibaba.com/self/visitors.htm'
css_tbody = '#J-visitors-tbl-tbody'
browser = alibaba_eyelash.browser

browser.get(url)
webdriver.ActionChains(browser).send_keys(Keys.ESCAPE).perform()
WebDriverWait(browser, 15).until(EC.visibility_of_element_located((By.CSS_SELECTOR, css_tbody)))

<selenium.webdriver.remote.webelement.WebElement (session="263f91b17375249705f15b47af27a5f1", element="0.23244758982830382-1")>

In [58]:
def next_page():
    css_pagination = '#J-pagination-visitors'
    css_next_btn = '#J-pagination-visitors span.ui-pagination-active+a'
    
    pagination = browser.find_element_by_css_selector(css_pagination)
    ActionChains(browser).move_to_element(pagination).perform()
    next_btn = browser.find_elements_by_css_selector(css_next_btn)
    if not next_btn:
        return False
    else:
        next_btn[0].click()
        WebDriverWait(browser, 15).until(EC.visibility_of_element_located((By.CSS_SELECTOR, css_tbody)))
        return True

In [61]:
def get_visitors_of_current_page():
    css_current_page = '#J-pagination-visitors span.ui-pagination-active'
    current_page_number = browser.find_element_by_css_selector(css_current_page).text
    
    visitors = []
    trs = browser.find_elements_by_css_selector(css_tbody+' tr.J-visitors-table-tr')
    for tr in trs:
        ActionChains(browser).move_to_element(tr).perform()
        visitor = {}
        visitor['date'] = tr.find_element_by_css_selector('td.td-checkbox input').get_attribute('statdate')
        visitor['id'] = tr.find_element_by_css_selector('td.td-visitor a').text
        visitor['region'] = tr.find_element_by_css_selector('td.td-region span').get_attribute('title')
        pv = tr.find_element_by_css_selector('td.td-pv span')
        visitor['pv'] = pv.text
        visitor['pv_detail'] = []
        
        if visitor['pv'] != '0':
            pv.click()
            pvd = WebDriverWait(browser, 15).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#J-visitor-detail')))
            close_btn = pvd.find_element_by_css_selector('span.vistor-detail-close')

            css_pv_detail_tbody = "#J-visitor-detail-tbl-tbody"
            WebDriverWait(browser, 15).until(EC.visibility_of_element_located((By.CSS_SELECTOR, css_pv_detail_tbody)))
            while True:
                pv_trs = browser.find_elements_by_css_selector(css_pv_detail_tbody+' tr')
                for pv_tr in pv_trs:
                    item = {}
                    item['idx'] = pv_tr.find_element_by_css_selector('td.visitor-detail-index').text
                    item['page'] = pv_tr.find_element_by_css_selector('td.visitor-detail-page a.J-visited-url-a').get_attribute('href')
                    item['stay'] = pv_tr.find_element_by_css_selector('td.visitor-detail-stay').text
                    item['time'] = pv_tr.find_element_by_css_selector('td.visitor-detail-time').text
                    visitor['pv_detail'].append(item)
                next_btn = browser.find_elements_by_css_selector('#J-pagination-visitor-detail a.ui-pagination-next')
                if next_btn:
                    next_btn[0].click()
                    WebDriverWait(browser, 15).until(EC.visibility_of_element_located((By.CSS_SELECTOR, css_pv_detail_tbody)))
                else:
                    close_btn.click()
                    WebDriverWait(browser, 15).until(EC.invisibility_of_element_located((By.CSS_SELECTOR, css_pv_detail_tbody)))
                    break
                    
        visitor['stay_duration'] = tr.find_element_by_css_selector('td.td-stay-duration').text
        visitor['stay_minisite_activity'] = []
        for span in tr.find_elements_by_css_selector('td.td-minisite-active span'):
            visitor['stay_minisite_activity'].append(span.get_attribute('textContent'))
        visitor['stay_website_activity'] = []
        for span in tr.find_elements_by_css_selector('td.td-website-active span'):
            visitor['stay_website_activity'].append(span.get_attribute('textContent'))
        visitors.append(visitor)
        
    return visitors
#     print(current_page_number, visitors)

In [60]:
visitor_list = []
while True:
    visitor_list = visitor_list + get_visitors_of_current_page()
    if not next_page():
        break
len(visitor_list)

590

In [62]:
%store visitor_list

Stored 'visitor_list' (list)


In [67]:
visitor_list[0]

{'date': '2018-09-04',
 'id': 'D1809041',
 'region': 'Canada',
 'pv': '3',
 'pv_detail': [{'idx': '1',
   'page': 'http://www.alibaba.com/product-detail/New-Styles-Super-Soft-Strip-False_60770970038.html?spm=a2700.7724857.normalList.137.c9947978sNcG4k',
   'stay': '7s',
   'time': '2018-09-04 23:53:14'},
  {'idx': '2',
   'page': 'http://glittereyelash.en.alibaba.com/productgrouplist-806057429/3D_Silk_Eyelashes.html?spm=a2700.details.pronpeci14.2.423f30e9VJc2L9',
   'stay': '287s',
   'time': '2018-09-04 23:48:27'},
  {'idx': '3',
   'page': 'http://www.alibaba.com/product-detail/New-Styles-Super-Soft-Strip-False_60770970038.html?spm=a2700.7724857.normalList.137.c9947978sNcG4k',
   'stay': '457s',
   'time': '2018-09-04 23:40:50'}],
 'stay_duration': '751s',
 'stay_minisite_activity': [],
 'stay_website_activity': ['总浏览量20', '浏览8个供应商']}

In [88]:
for v in visitor_list:
    if not v['date'].startswith('2018-09-04'):
        continue
    if v['region'].lower() == 'china':
        continue
    
    for pv in v['pv_detail']:
        href = re.sub('\?.*$', '',pv['page'])
        dt = pendulum.parse(pv['time']+'-07:00')
        print(pv['time'], dt, dt.in_timezone('Asia/Shanghai'))
    print(v['date'], v['region'])

2018-09-04 23:53:14 2018-09-04T23:53:14-07:00 2018-09-05T14:53:14+08:00
2018-09-04 23:48:27 2018-09-04T23:48:27-07:00 2018-09-05T14:48:27+08:00
2018-09-04 23:40:50 2018-09-04T23:40:50-07:00 2018-09-05T14:40:50+08:00
2018-09-04 Canada
2018-09-04 19:57:58 2018-09-04T19:57:58-07:00 2018-09-05T10:57:58+08:00
2018-09-04 Peru
2018-09-04 16:54:30 2018-09-04T16:54:30-07:00 2018-09-05T07:54:30+08:00
2018-09-04 16:54:52 2018-09-04T16:54:52-07:00 2018-09-05T07:54:52+08:00
2018-09-04 United States
2018-09-04 16:21:36 2018-09-04T16:21:36-07:00 2018-09-05T07:21:36+08:00
2018-09-04 16:21:28 2018-09-04T16:21:28-07:00 2018-09-05T07:21:28+08:00
2018-09-04 16:20:51 2018-09-04T16:20:51-07:00 2018-09-05T07:20:51+08:00
2018-09-04 16:20:17 2018-09-04T16:20:17-07:00 2018-09-05T07:20:17+08:00
2018-09-04 16:20:33 2018-09-04T16:20:33-07:00 2018-09-05T07:20:33+08:00
2018-09-04 16:19:56 2018-09-04T16:19:56-07:00 2018-09-05T07:19:56+08:00
2018-09-04 16:19:32 2018-09-04T16:19:32-07:00 2018-09-05T07:19:32+08:00
2018-